In [1]:
# импорт библиотек
import pandas as pd
import os
import requests
from requests.exceptions import ConnectionError
from time import sleep
import json
import ydata_profiling as yp
import sqlalchemy
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib_inline

In [3]:
# Входные данные
token = os.environ['token']
goals = os.environ['goals']
clientLogin = os.environ['clientLogin']
ReportsURL = os.environ['ReportsURL']

In [4]:
# HTTP-заголовки запроса, из справки api яндекс директа
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
           "Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto",
           # Формат денежных значений в отчете
           "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
           # "skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
           "skipReportSummary": "true"
           }

In [18]:
# Тело запроса, из справки api яндекс директа
body = {
    "params": {
        "SelectionCriteria": {
            "DateFrom": "2024-03-01",
            "DateTo": "2025-03-31"
        },
        "FieldNames": [
            "Date",            
            "TargetingLocationName",
            "Impressions",
            "Clicks",
            "Cost",
            "Conversions"
        ],
        "Goals": goals,
        "AttributionModels": ["AUTO"],
        "ReportName": "regions_stat",
        "ReportType": "CUSTOM_REPORT",
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",
        "IncludeVAT": "YES",
        "Page": {"Limit": 200000}
    }
}
# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)

In [19]:
# Запуск цикла для выполнения запросов
while True:
    try:
        req = requests.post(ReportsURL, body, headers=headers)
        req.encoding = 'utf-8'
        if req.status_code == 400:
            print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            break
        elif req.status_code == 200:
            print("Отчет создан успешно")
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            response = req.text
            break
        elif req.status_code == 201:
            print("Отчет успешно поставлен в очередь в режиме офлайн")
            retryIn = int(req.headers.get("retryIn", 60))
            print("Повторная отправка запроса через {} секунд".format(retryIn))
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            sleep(retryIn)
        elif req.status_code == 202:
            print("Отчет формируется в режиме офлайн")
            retryIn = int(req.headers.get("retryIn", 60))
            print("Повторная отправка запроса через {} секунд".format(retryIn))
            print("RequestId:  {}".format(req.headers.get("RequestId", False)))
            sleep(retryIn)
        elif req.status_code == 500:
            print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            
            break
        elif req.status_code == 502:
            print("Время формирования отчета превысило серверное ограничение.")
            print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
            print("JSON-код запроса: {}".format(body))
            print("RequestId: {}".format(req.headers.get("RequestId", False)))
            
            break
        else:
            print("Произошла непредвиденная ошибка")
            print("RequestId:  {}".format(req.headers.get("RequestId", False)))
            print("JSON-код запроса: {}".format(body))
            
            break

    # Обработка ошибки, если не удалось соединиться с сервером API Директа
    except ConnectionError:        
        print("Произошла ошибка соединения с сервером API")        
        break    

Параметры запроса указаны неверно или достигнут лимит отчетов в очереди
RequestId: 6014445452530612792


In [ ]:
response

In [ ]:
# Преобразование tsv-формата в json
tsv_data = response

# Разбиваем на строки
lines = tsv_data.split('\n')

# Получаем заголовки
headers = lines[0].split('\t')

result = []
for line in lines[1:]:
    if not line.strip():  # Пропускаем пустые строки
        continue
    
    values = line.split('\t')
    
    # Создаем словарь для каждой строки
    row_dict = {}
    for i, header in enumerate(headers):
        value = values[i] if i < len(values) else None
        
        # Преобразуем типы данных
        try:            
            if header in ['Impressions', 'Clicks']:
                value = int(value) if value != '--' else 0
            elif header == 'Cost':
                value = float(value.replace(',', '.')) if value != '--' else 0.0
            elif header == 'Conversions_252790207_AUTO':
                value = int(value.replace('--', '0')) if value != '--' else 0
        except (ValueError, AttributeError):
            value = values[i]  # Оставляем как есть, если преобразование не удалось
        
        row_dict[header] = value
    
    result.append(row_dict)

# Конвертируем в JSON
json_data = json.dumps(result, ensure_ascii=False, indent=2)
print(json_data)

In [ ]:
# Создание датафрейма
json_data = json.loads(json_data)
df = pd.DataFrame(json_data)

In [ ]:
df

In [ ]:
df.Date = df.Date.apply(pd.to_datetime, format='%Y-%m-%d').dt.date

In [ ]:
df.profile_report()

In [ ]:
#создаем подключение к базе Postgres на удаленном сервере

HOST = os.environ['HOST']
PORT = os.environ['PORT']
DATABASE = os.environ['DATABASE']
LOGIN = os.environ['LOGIN']
PASSWORD = os.environ['PASSWORD']

engine = sqlalchemy.create_engine(f'postgresql://{LOGIN}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')

In [ ]:
#загружаем датафрейм в базу данных

table_name = 'marketing_metrics'
schema_name = 'ormed'

df.to_sql(con=engine, name = table_name, schema=schema_name, index=False, if_exists='replace')